In [ ]:
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from MetaWorld.searchTest.utils import MyEnv, ToyExpertModel, sample_expert_transitions, benchmark_policy, LearnWrapper, train_policy, SuperMyGymWrapper, supermipWrapper, LearnWrapper
import torch
import zipfile
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import importlib
import numpy as np
from RlBaselines3Zoo import enjoy
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper

from stable_baselines3.common.policies import MultiInputActorCriticPolicy
from stable_baselines3.common.torch_layers import (
    BaseFeaturesExtractor,
    CombinedExtractor,
    FlattenExtractor,
    NatureCNN,
    create_mlp,
    get_actor_critic_arch,
)
from sb3_contrib.tqc.tqc import TQC
from sb3_contrib.tqc.policies import MultiInputPolicy
import numpy as np
import torch as th

model = enjoy.main(inpt_args="--algo tqc --env FetchPickAndPlace-v1 --folder /home/hendrik/Documents/master_project/Code/RlBaselines3Zoo/rl-trained-agents -n 300 --ret_model True")

In [ ]:
class TP_Wrapper(PPO):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def learn_fct(self, n_epochs):
        self.learn(total_timesteps=n_epochs)

class SAC_Wrapper(SAC):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def learn_fct(self, n_epochs):
        self.learn(total_timesteps=n_epochs)
    


In [ ]:
continue_policy = MultiInputActorCriticPolicy(observation_space=model.observation_space, action_space=model.action_space, lr_schedule=lambda a: 1, features_extractor_class=CombinedExtractor, net_arch=[512, 512, 512], )

In [ ]:
mon_env_wrapper = SuperMyGymWrapper(tag = None, bo=model.env)

In [ ]:
mon_env = mon_env_wrapper.make_wrapper(count_resets=True)

In [ ]:
mon_env.reset_count()

In [ ]:
continue_trainer = SAC_Wrapper(policy=MultiInputPolicy, env=model.env, policy_kwargs=dict(features_extractor_class=CombinedExtractor, net_arch=[512, 512, 512]), learning_rate=1e-3)

In [ ]:
state_dict = torch.load('/home/hendrik/Documents/master_project/LokalData/ImitationLearning/BC Trainer Large Model/best_modeltensor(-0.2800)')

In [ ]:
state_dict = torch.load('/home/hendrik/Documents/master_project/LokalData/ImitationLearning/BC MultiInputPolicy 100/best_modeltensor(-0.3200)')

In [ ]:
continue_trainer.policy.load_state_dict(state_dict)

In [ ]:
policy_trainer_wrapped = TP_Wrapper(policy='MultiInputPolicy', env=model.env, learning_rate=1e-3, policy_kwargs = dict(net_arch=[512, 512, 512]))

In [ ]:
policy_trainer_wrapped.policy.load_state_dict(state_dict)

In [ ]:
train_policy(
    trainer=continue_trainer,
    learn_fct=continue_trainer.learn_fct,
    val_env=model.env,
    logname='Continue SAC 100',
    path='/home/hendrik/Documents/master_project/LokalData/ImitationLearning/',
    n_epochs=1000,
    n_steps=50,
    eval_epochs=100,
    step_fct=lambda i: i+1
)

In [ ]:
global NUM_RESETS

In [ ]:
sd = torch.load('/home/hendrik/Documents/master_project/LokalData/ImitationLearning/BC Trainer Large Model/last_model')

In [ ]:
continue_trainer.policy.load_state_dict(sd)

In [ ]:
benchmark_policy(policy=continue_trainer.policy, path='/home/hendrik/Documents/master_project/LokalData/ImitationLearning/load_plicy/', logname='best val policy SAC', eval_epochs=500, val_env=model.env, stepid=1, best_reward=-100, save_model=False)

In [ ]:
torch.save(continue_trainer.policy.state_dict(), '/home/hendrik/Documents/master_project/LokalData/ImitationLearning/BC Trainer Large Model/last_model')